In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
import warnings
warnings.filterwarnings('ignore')
import urllib 
from decimal import Decimal
#import geojson
#import gdal, subprocess
import seaborn as sns
#import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy
import folium

In [ ]:
data = pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
data['Date'] = pd.to_datetime(data['Date'])
data_dateform = data
focuscrimes = (['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT'])

# Part 1: Temporal Patterns

Weekly Patterns

In [ ]:
data = data_dateform
weekdays = pd.Series(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
k = 0
fig = plt.figure(figsize=(25, 38))  

for foc in focuscrimes:#[0:4]
    sum_week = pd.DataFrame(index = weekdays, columns = ['total'])
    for i in weekdays:
        sum_week.loc[i,'total'] = len(data[(data['DayOfWeek']==i) & (data['Category']==foc)])
    k = k+1
    ax = fig.add_subplot(7, 2, k, frameon=True)
    ax.bar(sum_week.index,sum_week['total'])
    ax.set_xlabel('weekdays')
    ax.set_ylabel('crime amount')
    ax.set_title(foc)

Prostitution usually happens in the midweek and reached a peak on Thursday, while the crime amount were relatively low on Sunday and  Monday.

The months

In [ ]:
data['Month'] = data['Date'].dt.month
data['Count'] = np.ones([len(data),1])
fig = plt.figure(figsize=(25, 38))  
k = 1
for foc in focuscrimes:
    ax = fig.add_subplot(7,2,k)
    k += 1
    data_temp = data[data['Category']==foc]
    sum_month = data_temp.groupby('Month').sum()
    ax.bar(sum_month.index,sum_month['Count'])
    ax.set_xlabel('Month')
    ax.set_ylabel('crime amount')
    ax.set_title(foc)
    ax.set_xticks(np.linspace(1,12,12))

It seems that for most types of crimes, January and March are worse than other crimes, while February is better than January and March

The 24 hour cycle

In [ ]:
data['Time'] = pd.to_datetime(data['Time'])
data['Hours'] = data['Time'].dt.hour
data['Count'] = np.ones([len(data),1])
fig = plt.figure(figsize=(25, 40))  
k = 1
for foc in focuscrimes:
    ax = fig.add_subplot(7,2,k)
    k += 1
    data_temp = data[data['Category']==foc]
    sum_month = data_temp.groupby('Hours').sum()
    ax.bar(sum_month.index,sum_month['Count'])
    
    ax.set_xlabel('Hours')
    ax.set_ylabel('crime amount')
    ax.set_title(foc)
    ax.set_xticks(np.linspace(0,23,24))

For most types of crimes except prostitution, the crime amount reached a valley value at around 4am —— 6am. Most of the prostitution crimes happened in the evening or midnight.

Hours of the week

In [ ]:
weekdays = pd.Series(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
k = 1
for foc in focuscrimes:
    data_foc = data[data['Category']==foc]
    data_foc['Hours'] = data_foc['Time'].dt.hour
    data_foc['Count'] = np.ones([len(data_foc),1])
    fig = plt.figure(figsize=(30, 20))   
    ax = fig.add_subplot(7,2,k)
    k+=1
    for wd in weekdays:
        data_wd = (data_foc[data_foc['DayOfWeek']==wd])
        data_hours = data_wd.groupby('Hours').sum()
        if(wd =='Monday'):
            sum_hours = data_hours
        else:
            sum_hours = pd.concat([sum_hours,data_hours],axis = 0) 
    ax.bar(np.arange(0,168),sum_hours['Count'])
    ax.set_xlabel('Hours of the week')
    ax.set_ylabel('crime amount')
    ax.set_title(foc)
    ax.set_xticks(np.linspace(0,168,29))

From the hours of the week graph, the daily periodicity of each type of crime could be identified. However, for different types of crimes, the relative amount of crimes of each weekday are different. Some are more likely to happen around weekends, while some others are more likely to happen on workdays.

# Part 2: Thinking about data and visualization

**What is the difference between data and metadata? How does that relate to the bike-example?**

The function of metadata is to describe other data, which can make the processing of data much easier. It provides different types of information about a certain dataset, so that the person who is dealing with the dataset can take up the task from various perspective efficiently. Take the bike as an example, the size, weight, material, gear ratios and so on are as the metadata to describe the specific data/imformation of the bike.

**Sune says that the human eye is a great tool for data analysis. Do you agree? Explain why/why not. Mention something that the human eye is very good at. Can you think of something that is difficult for the human eye. Explain why your example is difficult.**

I agree that human eye is a great tool for data analysis. Human have gained great understanding of length, volumn, color, height from daily life. So as long as the data is presented in these forms by visualization, human can efficiently recognize the main idea in really short time, which is much faster than dealing with speech signals and other communication signals.
    
Besides, human eyes are really good at extracting typical character instead of the whole part of an object to help recognizing an     object rapidly.
    
Human eyes perform badly when they are trying to capture objects which move at extremely high speed. This task is difficult for human eye because the minimum response time for human eye is 0.1s. In this way, human eyes can't get precise details of fast-moving objects, therefore would lose some important information.

**Simpson's paradox is hard to explain. Come up with your own example - or find one on line.**

Simpson's paradox is a phenomenon in probability and statistics, in which a trend appears in several different groups of data but     disappears or reverses when these groups are combined.
    
Example: Batting averages of players in professional baseball. 
    
It is possible for one player to have a higher batting average than another player each year for a number of years, but to have a     lower batting average across all of those years. In both 1995 and 1996, Justice had a higher batting average than Jeter did. However, when the two baseball seasons are combined, Jeter shows a higher batting average than Justice. According to Ross, this phenomenon     would be observed about once per year among the possible pairs of players.
(Excerpted from Wikipedia)

**In your own words, explain the difference between exploratory and explanatory data analysis.**

Exploratory data analysis is the process that data analysts trying to discover the interesting points of a dataset. It includes coming up with assumption of the data, dealing with the data from different perspect to verify the assumption, creating value for a certain  trait of the data and using it to make improvement in some area.
    
Explanatory data analysis is a powerful tool to express a point of view and present a knowledge which is hard to describe in words. By analysing the data and visualizing it, a viewpoint could be explained precisely and efficiently, because human eyes are really good at extracting characteristics from visual information.

**As mentioned earlier, visualization is not the only way to test for correlation. We can (for example) calculate the Pearson correlation. Explain in your own words how the Pearson correlation works and write down it's mathematical formulation. Can you think of an example where it fails (and visualization works)?**

The Pearson correaltion is used to define the degree of correlation between two variables X and Y，which focus on linear correlation. The mathematical formulation shows as below:
$$ρ_{X,Y} = \frac{ cov(X,Y) }{σ_Xσ_Y}$$

Pearson correlation coefficient is equal to the covarianve of variable X and variable Y devided by standard deviation of X times      standard deviation of Y, and its value is between -1 and 1. If the value is negative, it means that there is a negative correlation   between these two variables and vice versa. And the larger absolute value of the coefficient is, the more significant the correlation between these two variables is.
    
One example that the Pearson correlation fails is when the correlation of two variables is not linear. Maybe the calculated Pearson   coefficient is equal to 0, but in fact there is a more complex correlation between them just instead of linear correlation. Another   example is related to outlier. For example, the serie of data is not linear correlated (different Y value but have the same value of X), however, if there is an outlier, it may cause the value of calculated Pearson correlation coefficient is very high. So in this    case, we need to do the visualization and it is not enough to only consider calculate the Pearson correlation coefficient. 

**What is the difference between a bar-chart and a histogram?**

Histogram: The horizontal axis of the histogram uses bins to divide the variable into a specific interval. Therefore, each interval in the histogram is next to each other, and each interval can be compared in size. The vertical axis of the histogram represents the number of variables. It also can show the distribution of dataset in a certain degree.
    
Bar-chart: An interval of a bar chart represents a categorical variable. The intervals do not need to be next to each other because they are parallel to each other and there is no comparison of sizes. The vertical axis of the bar chart can represent the number of variables or the frequency of the variables.
    
In general, histogram is to describe the numerical variables and show the trend of data variation but purpose of bar chart is describing the categorical variables and showing the difference between each category.

**I mention in the video that it's important to choose the right bin-size in histograms. But how do you do that? Do a Google search to find a criterion you like and explain it.**

We do a lot of search about how to choose the optimal number of the bin, but most of the results indicate that there is no best way to calculate the bin size and the existing methods require that the data should base on certain distribution. We choose the Sturges' formula as the one here to do some explaination. The Sturges' formula shows as below:

$$k = log_2^n+1$$
    
k is the number of bins and n is the number of data. "It implicitly bases the bin sizes on the range of the data and can perform poorly if n < 30, because the number of bins will be small—less than seven—and unlikely to show trends in the data well. It may also perform poorly if the data are not normally distributed."--Wikipedia 
    
So it still has some limination, we should try lots of methods and look though the results we get to decide the number of the bins.

# Part3: Generating important plot types

We choose the category `'LARCENY/THEFT'`, which occured in January and Feruary 2017

In [ ]:
crime=pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')
LarThef=crime[(crime['Category']== 'LARCENY/THEFT') ]
LarcThefJanFeb=LarThef[
      (pd.to_datetime(LarThef['Date']) >= pd.to_datetime('2017-01-01')) 
    & (pd.to_datetime(LarThef['Date']) <  pd.to_datetime('2017-03-01'))
]

We invetigate the crime happening from 13 to 14

In [ ]:
LarcThefJanFeb_time = LarcThefJanFeb[
    (pd.to_datetime(LarcThefJanFeb['Time'])>=pd.to_datetime('13:00')) 
    & (pd.to_datetime(LarcThefJanFeb['Time'])<=pd.to_datetime('14:00'))]

In [ ]:
fig,ax = plt.subplots(figsize=(28,10),dpi = 80)
plt.xticks(rotation=45,size=12)
plt.xlabel('xlabel',fontsize = 15)
plt.title("Jitter-plot: Registered time of Larceny/theft crimes in January and February 2017 between 13 and 14",size = 18)
sns.stripplot(LarcThefJanFeb_time.Time.sort_values(),jitter = 0.4,size = 10,ax = ax,linewidth =.8,color="red")


The jitter-plot shows that the happening rate of crime `'LARCENY/THEFT'` is higher at 13:00 and 14:00, 13:30 as well as 13:40.

The next step is to plot some histograms. From the data shown in exercise 2, it can be seen that the `'LARCENY/THEFT'` and `'VEHICLE THEFT'` are with different geographical patterns. We chooes the time period '2017-01-01' to '2017-03-01' for `'LARCENY/THEFT'` and period '2016-06-01' to '2017-06-01' for `'VEHICLE THEFT'`, so that both of the two dataset contain around 7000 data.

In [ ]:
VEHICLE_THEFT = crime[(crime['Category']== 'VEHICLE THEFT')]
VEHICLE_THEFT_16Jun_17Jun=VEHICLE_THEFT[
      (pd.to_datetime(VEHICLE_THEFT['Date']) >= pd.to_datetime('2016-06-01')) 
    & (pd.to_datetime(VEHICLE_THEFT['Date']) <  pd.to_datetime('2017-06-01'))]

In [ ]:
plt.figure(figsize=(20,5))
ax1 = plt.subplot(1,2,1)
ax1.hist(VEHICLE_THEFT_16Jun_17Jun.Y,bins = 50,edgecolor='black',linewidth =0.8)
ax1.set_xlabel('Latitude')
ax1.set_ylabel('Number of VEHICLE THEFT')
ax2 = plt.subplot(1,2,2)
ax2.hist(LarcThefJanFeb.Y,bins = 50,edgecolor='black',linewidth =0.8)
ax2.set_xlabel('Latitude')
ax2.set_ylabel('Number of ASSAULT')
plt.show()

These two histograms prove that the geographical patterns for the two type of crimes are totally different, as it can be found that the disribution for `'VEHICLE THEFT'` is more uniform compared to the one for `'LARCENY/THEFT'`. But both of them indicate that these crimes are more likely to occur around latitude 37.78.

Next up is to creat the kernel density plot.

In [ ]:
fig=plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(111)
ax1.hist(LarcThefJanFeb.Y,bins = 50,edgecolor='black',linewidth =0.8,color='w',linestyle = '--',label='Histogram')
ax1.legend(loc=2)
ax2 =ax1.twinx()
sns.kdeplot(np.array(LarcThefJanFeb.Y),bw=0.006,label='KDE,Banwidth=0.006')
sns.kdeplot(np.array(LarcThefJanFeb.Y),bw=0.002,label='KDE,Banwidth=0.002')
ax2.legend(loc=0)
plt.show()

From the plot, the shape of the kde plot depends on the parameter 'Bandwidth'. It is not difficult to find that if the the 'Bandwidth' we chosen is too large, then the estimated result will be underfit; Similary if the 'Bandwidth' is too small, it will overfit the data.
    
Grab 25 random timepoint form the dataset to check if they fit the KDEs.

In [ ]:
from random import sample
LarcThefJanFeb_sample_index = sample(LarcThefJanFeb.index.tolist(),25)
LarcThefJanFeb_sample = LarcThefJanFeb.loc[LarcThefJanFeb_sample_index,:]
fig = plt.figure(figsize=(8,5))
ax1 = fig.add_subplot(111)
ax1.hist(LarcThefJanFeb_sample.Y,bins = 50,edgecolor='black',linewidth =0.8,color='w',linestyle = '--',label='Histogram')
ax1.legend(loc=2)
ax2 =ax1.twinx()
sns.kdeplot(np.array(LarcThefJanFeb_sample.Y),bw=0.006,label='KDE,Banwidth=0.006')
sns.kdeplot(np.array(LarcThefJanFeb_sample.Y),bw=0.002,label='KDE,Banwidth=0.002')
ax2.legend(loc = 0)
plt.show()

The picture above shows significantly that the shape of KDE model are totaly different using the same 'Bandwith', which proves that the KDEs can be misleading as the distribution of the 25 random timepoints may not be at the same as the population. 

# Part 4: Heatmaps of geo-data

We look at studying geospatial data by plotting raw data points as well as heatmaps on top of actual maps

In [ ]:
map_hooray = folium.Map([37.77919, -122.41914], zoom_start=13)
folium.Marker([37.77919, -122.41914], popup='City Hall').add_to(map_hooray)
map_hooray

* Create a map of all arrests and draw a little dot for `'DRUG/NARCOTIC'` during June-July 2016. 

In [ ]:
DrugNarc = crime[(crime['Category']== 'DRUG/NARCOTIC') ]

In [ ]:
#Extract crimes only from first week of june 2016
DrugNarcJune=DrugNarc[
      (pd.to_datetime(DrugNarc['Date']) >= pd.to_datetime('2016-06-01')) 
    & (pd.to_datetime(DrugNarc['Date']) <= pd.to_datetime('2016-06-07'))
]


In [ ]:
#Create feature group for mapping 'DRUG/NARCOTIC' incidents.
DrugNarcJune_FG = folium.FeatureGroup(name="Drugs & Narcotics, first week of June 2016")

for index, row in DrugNarcJune.iterrows():                                    
    folium.CircleMarker([row['Y'], row['X']],
                        radius=2,
                        color='brown',
                       ).add_to(DrugNarcJune_FG)

In [ ]:
#Create map and add feature group
SFmapFeature=folium.Map([37.77919, -122.41914], zoom_start=12)
SFmapFeature.add_child(DrugNarcJune_FG)
SFmapFeature

* Create a heatmap of all arrests for `'SEX OFFENSES, NON FORCIBLE'` during June-July 2016.

In [ ]:
SexOff = crime[(crime['Category']== 'SEX OFFENSES, NON FORCIBLE') ]

In [ ]:
#Extract crimes only from 
SexOffJune=SexOff[
      (pd.to_datetime(SexOff['Date']) >= pd.to_datetime('2010-01-01')) 
    & (pd.to_datetime(SexOff['Date']) <= pd.to_datetime('2018-12-30'))
]

In [ ]:
#Converting locations to an array
SexOffJune_arr = SexOffJune[['Y', 'X']].values

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

SFmapSexOff=folium.Map([37.77919, -122.41914], zoom_start=12)
folium.Marker([37.77919, -122.41914], popup='City Hall').add_to(SFmapSexOff)
SFmapSexOff.add_child(plugins.HeatMap(SexOffJune_arr))
SFmapSexOff

**What can you see using the scatter-plots that you can't see using the heatmaps? And vice versa: what does the heatmaps help you see that's difficult to distinguish in the scatter-plots?** 

Heatmap can tell you the crime rate of these places through the intensity of color. On the other hand, it's easier to see the exact   location of the crime on street-level or house level.

**Comment on the effect on the various parameters for the heatmaps. How do they change the picture? (at least talk about the radius and max_zoom).**

Zoom_max determines zoom level where the points reach maximum intensity and Radius determines radius of each point of the heatmap.

* Exercise: Heat map movies. This exercise is a bit more independent than above - you get to make all the choices.

In [ ]:
LARCENY = crime[(crime['Category']=='LARCENY/THEFT') ]
LARCENY['Time'] = pd.DatetimeIndex(LARCENY['Time']).hour

In [ ]:
LARCENY_arr_p = LARCENY[['Y', 'X']].values
LARCENY_arr_time = LARCENY['Time'].values
LARCENY_arr_plist = LARCENY_arr_p.tolist()
LARCENY_list_time = list(range(24))

In [ ]:
list_all = []
list_time = []
for i in range(24):
    for index, row in LARCENY[(LARCENY['Time']==i)].iterrows():
        list_time.append([row['Y'], row['X']])
    list_all.append(list_time)
    list_time = []

In [ ]:
SFmapheatmoive=folium.Map([37.77919, -122.41914], zoom_start=12)
hm = plugins.HeatMapWithTime(
    list_all,
    index=LARCENY_list_time,
    auto_play=True,
    max_opacity=0.9
)

hm.add_to(SFmapheatmoive)

SFmapheatmoive

**Comment on your results:**
  * **What patterns does your movie reveal?**
  * **Motivate/explain the reasoning behind your choice of crimetype and time-resolution.**

This movie reveals the crime rate of larceny / theft every hour of the day. Crime rates are relatively high in the northeast of the city between 10 p.m. and 12 p.m. and between 10 a.m. and 13 a.m.
    
Larceny / theft is the most frequent crime type. We could analyze when and where the incidence is high in a day to prevent such crime.

# Part 5: Errors in the data. The importance of looking at raw (or close to raw) data

In each of the three examples above, describe in your own words how the data-errors I call attention to above can bias the binned versions of the data. Also briefly mention how not noticing these errors can result in misconceptions about the underlying patterns of what's going on in San Francisco (and our modeling).

**1.In the hourly activity for PROSTITUTION something surprising is going on on Thursday. Remind yourself here, where I've highlighted the phenomenon I'm talking about.**

It shows that there is an outlier on Thursday, if we bin the data with this outlier, it will show that the data on Thursday is extremely large which may not very reasonable. This outlier maybe wrong data or calculated wrongly.

**2.When we investigated the details of how the timestamps are recorded using jitter-plots, we saw that many more crimes were recorded e.g. on the hour, 15 minutes past the hour, and to a lesser in whole increments of 10 minutes. Crimes didn't appear to be recorded as frequently in between those round numbers. Remind yourself here, where I've highlighted the phenomenon I'm talking about.**

The recorder may not record accurately when recording the time of the crime. They may choose to record at a time point such as 10 minutes or 15 minutes. When a jitter plot of the arrest times during a single hour is created, we may make misconceptions that those crimes occurred less frequently in between those round numbers.

**3.And finally, today we saw that the Hall of Justice seemed to be an unlikely hotspot for sex offences. Remind yourself here.**

From the heatmap, it is possible to observe that the crime 'sex offense' occures frequently near the Hall of Justice. However, the Hall of Justice seems not to be a place that have high crime rate for sex offences. Since the heatmap based on the geographic coordinates of the crime, and the depth of the color is determined according to the density of the drawn coordinates. Perhaps the crime rate in the area around Hall of Justice is high, causing this area  to be marked as hotspot in the heatmap.